In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
import json

In [2]:
def Dorfman(model, where):

    if where == gp.GRB.Callback.MIPSOL:
        EQ = 0
        for s in escenarios:
            sub = gp.Model()
            sub.params.OutputFlag = 0

            x2 = sub.addVars(grupos, individuos, vtype=gp.GRB.BINARY, name="x2")
            y2 = sub.addVars(grupos, vtype=gp.GRB.BINARY, name="y2")
            
            # sub.addConstrs(sum(x2[i, j] for j in individuos) <= GT for i in grupos)
            sub.addConstrs(x2[i, j] >= model.cbGetSolution(x1[GT][i, j])*y2[i] + model.cbGetSolution(z1[GT][i])-1 for j in individuos for i in grupos)
            sub.addConstrs(y2[i]*1e6 >= sum(model.cbGetSolution(x1[GT][i, j]) * ret[s, j] for j in individuos) for i in grupos)

            sub.setObjective(sum(x2[i, j] for i in grupos for j in individuos), gp.GRB.MINIMIZE)

            sub.optimize()
            EQ += sub.objVal
        EQ = EQ/S
        model.cbLazy(EQ <= theta[GT])

In [3]:
with open("prevalencia.json", "r") as file:
    diccionario = json.load(file)

pDorfman = []
prevalencia = []
resultados = []
for a, b in diccionario.items():
    pDorfman.append(float(a))
    prevalencia.append(np.array(b["prevalencia"])[0:346])
    resultados.append(np.array(b["resultado"])[0:346])
pDorfman = np.array(pDorfman)
prevalencia = np.array(prevalencia) 
resultados = np.array(resultados)


In [4]:
I = prevalencia.shape[1] # Cantidad de personas
S = 5 # Cantidad de simulaciones
alpha = 5*I # Cantidad de test disponibles
beta = 0.2 # Cantidad de volumen de muestra que necesita un test
delta = 0.3 # Umbral para solo testear

individuos = range(I)
muestras = 2*np.ones(I)
escenarios = range(S)

# real_p = np.round(np.random.uniform(low=0.01, high=0.5, size=I), decimals=2)
# real_p = 
# p = np.mean(real_p)
# hat_p = np.ones(I)*p

In [5]:
for i in range(len(pDorfman)):
    p = pDorfman[i]
    hat_p = prevalencia[i]
    ret = np.array([[np.random.binomial(n=1, p=x) for x in hat_p] for _ in escenarios])
    
    val = []
    x1 = {}
    y1 = {}
    z1 = {}
    theta = {}
    master = {}
    for GT in range(4,8):
        print(GT)
        gsum = 0
        for j in range(len(hat_p)):
            if hat_p[i] >= 0.25:
                gsum += 1

        G = int(np.ceil(I/GT)) + gsum + 12
        grupos = range(G)
        
        L = 0
        
        master[GT] = gp.Model()
        master[GT].params.OutputFlag = 0
        master[GT].params.LazyConstraints = 1

        x1[GT] = master[GT].addVars(grupos, individuos, vtype=gp.GRB.BINARY, name="x1")
        y1[GT] = master[GT].addVars(grupos, vtype=gp.GRB.BINARY, name="y1")
        z1[GT] = master[GT].addVars(grupos, vtype=gp.GRB.BINARY, name="z1")
        theta[GT] = master[GT].addVar(lb=L)

        master[GT].addConstrs(y1[GT][i]*1e6 >= sum(x1[GT][i, j] for j in individuos) for i in grupos) #R1
        master[GT].addConstrs(z1[GT][i]*1e6 >= sum(x1[GT][i, j] for j in individuos) - 1 for i in grupos) #R2
        master[GT].addConstrs(sum(x1[GT][i, j] for i in grupos) == 1 for j in individuos) #R3
        # master[GT].addConstr(sum(y1[GT][i] for i in grupos) <= alpha) #R4
        # master[GT].addConstrs(sum(muestras[j]*x1[GT][i,j] for i in grupos) >= beta for j in individuos) #R5
        # master[GT].addConstrs((muestras[j] <= delta) >> (sum(x1[GT][i,j] for i in grupos) == 1) for j in individuos) #R6
        # master[GT].addConstrs((muestras[j] <= delta) >> (sum(x1[GT][i,j] for i in grupos) == 1) for j in individuos)

        master[GT].addConstrs(sum(x1[GT][i, j] for j in individuos) <= GT for i in grupos) #R7
        # master[GT].addConstr(sum(x1[GT][i, j] for i in grupos for j in individuos) >= I) #R8
        for j in range(len(hat_p)):
            if hat_p[i] >= 0.25:
                master[GT].addConstr((x1[i,j] == 1) >> (sum(x1[i,k] for k in individuos) == 1) for i in grupos)

        master[GT].setObjective(sum(y1[GT][j] for j in grupos) + theta[GT], gp.GRB.MINIMIZE)

        master[GT].optimize(Dorfman)
        val.append(master[GT].objVal)
    print(val)
    GTO = val.index(min(val)) + 4
    G = int(np.ceil(I/GTO)) + gsum + 12

    print(GTO)
    config = np.zeros((G, I))
    ik, jk = -1, -1
    for var in master[GTO].getVars():
        if "x1" in var.VarName:
            jk += 1
            if jk % I == 0:
                ik += 1
                jk = 0

            if var.X == 1:
                config[ik, jk] = 1
    plt.spy(config)
    plt.title(f"Distribución de personas con GT: {GTO}")
    plt.ylabel("Grupos")
    plt.xlabel("Individuos")
    plt.show()

    plt.plot(range(1, len(val)+1), val)
    plt.scatter(range(1, len(val)+1), val, label="Costo Esperado")
    plt.title(f"Prevalencia: {p},    Opt GT: {GTO},    Costo Esperado: {master[GTO].objVal:0.2f}")
    plt.xlabel("Group Size")
    plt.ylabel("Costo")
    plt.legend()
    plt.show()

    
    # Se teste el rendimiento de esta configuración usando las prevalencias reales 
    gamma = 0
    for temp in range(config.shape[0]):
        if np.sum(config[temp,:]) > 0:
            gamma += 1
    for t in resultados[i,:]:
        if t == 1:
            gamma += sum(config[t,:])
    print(f"El resultado final usando la configuración es: {gamma}")

4
Academic license - for non-commercial use only - expires 2023-06-21
Using license file C:\Users\jacmo\gurobi.lic


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "src\gurobipy\callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "C:\Users\jacmo\AppData\Local\Temp\ipykernel_11292\1733593922.py", line 1, in Dorfman
KeyboardInterrupt: 
